In [1]:
from datasets import load_dataset

raw_dataset = load_dataset("conll2003")
dataset_feature = raw_dataset["train"].features
ner_labels = dataset_feature["ner_tags"].feature.names
id2label = {str(i): label for i, label in enumerate(ner_labels)}
label2id = {value: key for key, value in id2label.items()}

Found cached dataset conll2003 (C:/Users/zimin/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/9a4d16a94f8674ba3466315300359b0acd891b68b6c8743ddf60b9c702adce98)


  0%|          | 0/3 [00:00<?, ?it/s]

In [2]:
from transformers import AutoTokenizer
model_checkpoint = "dslim/bert-base-NER"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

OSError: dslim/bert-base-NER does not appear to have a file named config.json. Checkout 'https://huggingface.co/dslim/bert-base-NER/None' for available files.

In [ ]:
from transformers import AutoTokenizer
model_checkpoint = "dslim/bert-base-NER"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
batch_size = 32


from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
from transformers import DataCollatorForTokenClassification
from datasets import load_metric

model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint, 
    num_labels=len(ner_labels),
    id2label = id2label,
    label2id = label2id
)


args = TrainingArguments(
    model_checkpoint,
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01
)

data_collator = DataCollatorForTokenClassification(tokenizer)
metric = load_metric("seqeval")


In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset = tokenized_dataset["train"],
    eval_dataset = tokenized_dataset["validation"],
    data_collator = data_collator,
    tokenizer = tokenizer,
    compute_metrics = compute_metrics
)

trainer.train() 


